# Example: Working with SPEAR Forecast Data

The real-time SPEAR forecast data are delivered as one file per varaible, per ensemble member. This notebook illustrates how to concatenate the individual files into a single dataset with an `ensemble` dimension. Extracting data at individual tige gauge locations is also demonstrated.

In [1]:
import itertools
import numpy as np
import xarray as xr
import momlevel
from pathlib import Path

## Forecast and Data Parameters

In [2]:
# Forecast parameters
rootdir = Path("/net2/jpk/NMME_sealevel/september_retrospective_SPEAR_MED")
date = "20220901"

# Ocean static file (contains grid information)
static = rootdir / "ocean.static.nc"

# Output directory (default is `rootdir`)
outdir = rootdir

In [3]:
# List of ensemble members (currently n=30)
enslist = list(range(1, 31))

# List of sea level fields
varlist = ["hfds", "net_heat_surface", "PRCmE", "slp", "ssh", "SST", "taux", "tauy"]

In [4]:
# Construct a dictionary of all expected files

filedict = {
    var: [rootdir / f"{date}.{var}_ens{str(ens).zfill(2)}.nc" for ens in enslist]
    for var in varlist
}

## Confirm Expected Data Exists

In [5]:
# Master list of all files
allfiles = list(itertools.chain(*filedict.values()))
allfiles.append(static)

In [6]:
# Determine status of each file
status = [x.exists() for x in allfiles]

In [7]:
# Report back the status of any missing files

try:
    assert all(x is True for x in status)

except AssertionError as exc:
    missing = [
        filename for exists, filename in zip(status, allfiles) if exists is False
    ]
    raise FileNotFoundError(f"Unable to locate file(s): {missing}")

## Combine Into Single Gridded Dataset

In [8]:
# Generate an xarray dataset for each variable with a new `ensemble` dimension

filedict = {
    k: xr.open_mfdataset(v, combine="nested", concat_dim="ensemble", use_cftime=True)
    for k, v in filedict.items()
}

In [9]:
# Merge datasets for individual fields
dset = xr.merge(list(filedict.values()), compat="override")

In [10]:
# Merge static data
static = xr.open_dataset(rootdir / "ocean.static.nc", decode_times=False)
dset = xr.merge([dset, static], compat="override")

In [11]:
# Ensure that `time` is the first dimension
dset = dset.transpose("time", ...)

In [12]:
# Write out gridded file
dset.to_netcdf(outdir / f"SPEAR_sealevel_forecast_{date}.gridded.nc")

## Extract Tide Gauge Time Series

In [13]:
# Nominal resolution, coverted to km
print(f"Nominal resolution: {float((dset.dyt.mean()/1000.))} km")

Nominal resolution: 55.08805078125 km


In [14]:
# Consider ocean points that lie within a 55-km radius (~34 miles) 
# of the real world tide gauge location

ds_tg = momlevel.extract_tidegauge(
    dset.ssh, dset.geolon, dset.geolat, mask=dset.wet, threshold=55.0
)

In [15]:
# Write out tide gauge file
ds_tg.to_netcdf(outdir/f"SPEAR_sealevel_forecast_{date}.tidegauge.nc")